In [4]:
# import libraries
import os
from dotenv import load_dotenv
from groq import Groq

# load environment variables
load_dotenv()

# create client calling Groq class
client = Groq(api_key=os.getenv('GROQ_API_KEY'))

question ="how to reduce weight"

# create a query
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content":question,
        }
    ],
    model="llama3-8b-8192",
)

# print the response
print(response.choices[0].message.content)

Reducing weight, also known as weight loss, is a complex process that requires patience, persistence, and a combination of lifestyle changes. Here are some effective ways to reduce weight:

**1. Create a calorie deficit**: Reduce your daily caloric intake by eating fewer calories than your body burns. Aim for a deficit of 500-1000 calories per day for safe and sustainable weight loss.

**2. Eat a healthy and balanced diet**: Focus on whole, unprocessed foods like vegetables, fruits, whole grains, lean proteins, and healthy fats. Avoid sugary drinks, fast food, and processed snacks.

**3. Incorporate physical activity**: Regular exercise not only burns calories but also builds muscle mass, which further boosts metabolism. Aim for at least 150 minutes of moderate-intensity aerobic exercise, or 75 minutes of vigorous-intensity aerobic exercise, or a combination of both, per week.

**4. Strength training**: Building muscle through strength training exercises can help increase your resting 

In [5]:
# In this section we use a simple inbuilt search engine
# Download the necessary files using wget
# use this search engine to take data from FAQ document and put the data to search engine and perform a simple search 
#downloading the search file raw file from github
import wget

url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py'
wget.download(url)

# below is the FAQ document parsed in json structure for extreacting data
url2 = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json'
wget.download(url2)

100% [.................................] 658332 / 658332

'documents (1).json'

In [6]:
import os
import json
from dotenv import load_dotenv
import minsearch
from groq import Groq
load_dotenv()

with open("documents.json",'rt') as f:
  docs_raw = json.load(f)
documents = []
for course_dict in docs_raw:
  for doc in course_dict['documents']:
    doc['course'] = course_dict['course']
    documents.append(doc)

#creating index, segregating data which are text fields and which are keyfields- 
#text_fields are performance 
#keyfields to what exactly we want to search here it is data-engineering zoomcamp
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)


In [7]:

query = 'When is the course starting'

boost = {'question': 3.0, 'section': 0.5}

results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )


In [8]:
# prompt template is for giving a role, good practice for better response, for more advacne model we need it


def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    QUESTION: {question}
    CONTEXT: 
    {context}
    """.strip()

    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

prompt = build_prompt(query,results)

#client = OpenAI(api_key=os.getenv('OPEN_API_KEY'))
client = Groq(api_key=os.getenv('GROQ_API_KEY'))

response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages = [{"role":"user","content":prompt}])

print(response.choices[0].message.content)

Based on the CONTEXT, the answer to the QUESTION "When is the course starting?" is:

The course will start on 15th January 2024 at 17h00 with the first "Office Hours" live session.


In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
#using elastic search
from elasticsearch import Elasticsearch

In [11]:
es_client = Elasticsearch('http://localhost:9200')

In [12]:
es_client.info()

ObjectApiResponse({'name': '1ad7182d536b', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'hNSSd0WEQFOsviyTKBNmdQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [15]:
#index in elasticsearch is like a table in Database
#syntax is a bit complicated

index_settings= {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [16]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

D:\Projects\LLM_ChatBot\llmenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|███████████████████████████████████| 948/948 [00:55<00:00, 17.11it/s]


In [37]:
query = "i just started the course can ijoin the course"



In [38]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [43]:
response = es_client.search(index=index_name,q=query)
response

ObjectApiResponse({'took': 29, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1764, 'relation': 'eq'}, 'max_score': 16.529968, 'hits': [{'_index': 'course-questions', '_id': 'vSyGSJABrMKIDhBCTslM', '_score': 16.529968, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local machine to docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'cSyZSJABrMKIDhBC6M2a', '_score': 16.529968, '_source': {'text': "You can copy files from your local machine i

In [40]:
query = "i just started the cours,e can ijoin the course"
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])


In [41]:

client = Groq(api_key=os.getenv('GROQ_API_KEY'))

response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages = [{"role":"user","content":prompt}])



print(response.choices[0].message.content)

To execute a command in a running Docker container, you can use the `docker exec` command. The basic syntax is as follows:

`docker exec -it <container-id> bash`

This will open a new shell within the specified container.


In [69]:
##### Homework ########


import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [70]:
query =  "How do I execute a command in a running docker container?"

In [72]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [73]:
response = es_client.search(index=index_name,q=query)
response

ObjectApiResponse({'took': 17, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1764, 'relation': 'eq'}, 'max_score': 16.529968, 'hits': [{'_index': 'course-questions', '_id': 'vSyGSJABrMKIDhBCTslM', '_score': 16.529968, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local machine to docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'cSyZSJABrMKIDhBC6M2a', '_score': 16.529968, '_source': {'text': "You can copy files from your local machine i

In [74]:
response['hits']['hits']

[{'_index': 'course-questions',
  '_id': 'vSyGSJABrMKIDhBCTslM',
  '_score': 16.529968,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I copy files from my local machine to docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course-questions',
  '_id': 'cSyZSJABrMKIDhBC6M2a',
  '_score': 16.529968,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker 

In [ ]:
context_template = """
Q: {question}
A: {text}
""".strip()